In [134]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [135]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
filename = os.path.join(os.getcwd(), "data", "postings500.csv")
df = pd.read_csv(filename, header=0)
df.head()

,JobDescription,Categorized_Posting
0,SURVIVABILITY ENGINEER 2\n\nLocation:\n\nNewpo...,"```json\n{\n ""Marketing"": [\n ""Want to be ..."
1,Requisition Number: 15838 \n\nRequired Travel:...,"```json\n{\n ""Marketing"": [\n ""HII b..."
2,"**Secure our Nation, Ignite your Future**\n\n*...","```json\n{\n ""Marketing"": [\n ""Secur..."
3,Description:\n\nWe are looking for an Instrume...,"```json\n{\n ""Marketing"": [\n ""At Ac..."
4,RH2 is currently seeking an entry-level Staff ...,"```json\n{\n ""Marketing"": [\n ""Our team is..."


In [5]:
def extract_marketing(posting):
    posting = json.loads(df['Categorized_Posting'][0][7:-3])
    return posting['Marketing']

def extract_legal(posting):
    posting = json.loads(df['Categorized_Posting'][0][7:-3])
    return posting['Legal']

def extract_description(posting):
    posting = json.loads(df['Categorized_Posting'][0][7:-3])
    return posting['Description']

def extract_requirements(posting):
    posting = json.loads(df['Categorized_Posting'][0][7:-3])
    return posting['Requirements']
    
    
df['Description'] = df['Categorized_Posting'].apply(extract_description)
df['Requirements'] = df['Categorized_Posting'].apply(extract_requirements)
df['Legal'] = df['Categorized_Posting'].apply(extract_legal)
df['Marketing'] = df['Categorized_Posting'].apply(extract_marketing)

new_rows = []

for _, row in df.iterrows():
    for sentence in row['Description']:
        new_rows.append({'Sentence': sentence, 'Label': '__label__Description'})
    for sentence in row['Requirements']:
        new_rows.append({'Sentence': sentence, 'Label': '__label__Not_Description'})
    for sentence in row['Legal']:
        new_rows.append({'Sentence': sentence, 'Label': '__label__Not_Description'})
    for sentence in row['Marketing']:
        new_rows.append({'Sentence': sentence, 'Label': '__label__Not_Description'})

df_sentences = pd.DataFrame(new_rows)
df_sentences.head()

,Sentence,Label
0,"With more than 25,000 employees - including th...",__label__Description
1,Our diverse and innovative team of professiona...,__label__Description
2,"Anchored in our rich, 135-year history, we col...",__label__Description
3,This position is in the Ship Survivability Dep...,__label__Description
4,The successful applicant should have technical...,__label__Description


In [13]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_sentences, test_size=0.33, random_state=42)

with open('train.txt', 'w') as f:
    for index, row in train_df.iterrows():
        f.write(f"{row['Label']} {row['Sentence']}\n")

# Save the test data
with open('test.txt', 'w') as f:
    for index, row in test_df.iterrows():
        f.write(f"{row['Label']} {row['Sentence']}\n")

In [14]:
!pip install --upgrade numpy fasttext-wheel

In [15]:
import fasttext

model = fasttext.train_supervised(input="train.txt", epoch=25, lr=1.0, wordNgrams=2)

model.save_model("job_postings_classifier.ftz")

In [16]:
model.test("test.txt")

(4125, 1.0, 1.0)

In [17]:
sentences = ["U.S. Citizenship may be required for certain positions.", "Master's Degree and 0 year of relevant professional experience."]
predictions = model.predict(sentences)

print(predictions)

([['__label__Not_Description'], ['__label__Not_Description']], [array([1.0000081], dtype=float32), array([1.0000052], dtype=float32)])


In [29]:
filename_data = os.path.join(os.getcwd(), "data", "Engineer_20230826.csv")
df_jobs = pd.read_csv(filename_data, header=0)
df_jobs.head()

def extract_descriptions(job_description):
    sentences = job_description.split('. ') 
    description_sentences = []
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    labels = model.predict(sentences)[0] 

    for sentence, label in zip(sentences, labels):
        if label[0] == '__label__Description': 
            description_sentences.append(sentence.replace('\\n', '\n'))
    
    return " ".join(description_sentences)

df_jobs['ExtractedDescription'] = df_jobs['JobDescription'].apply(extract_descriptions)


df_jobs = df_jobs.drop(['OrigJobTitle', 'JobTitle', 'JobDescription'], axis=1)
df_jobs.head()
df_jobs.to_csv('data/description_extracted.csv', index=False)


In [31]:
for value in df_jobs["ExtractedDescription"]:
    print(value)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




Our office provides support for other Jacobs offices and regions, so there is potential opportunity for work on major, exciting projects across the USA and internationally.

**Principal Duties and Responsibilities:**

+ Perform transportation/traffic engineering planning and design including managing tasks and projects and quality control.

+ Function as a technical specialist formulating and developing advanced engineering concepts and techniques.

+ Perform civil design calculations, sketches, drawings, technical comparisons and similar technical work as required by assignments.

+ Develop solutions to complex problems that require a high degree of innovation and ingenuity.

+ Participate in planning, cost development and management, and scheduling for assigned projects.

+ Maintain effective communication with project team members and with clients.

+ Research traffic analysis, traffic operations, and traffic safety issues and communicate Jacobs innovative approach to City, State, 